In [ ]:
import pymongo, itertools, collections

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017')
players, matches = client['usta'].players, client['usta'].matches
print(players.count(), matches.count())

In [ ]:
def get_total_games(score):
    return sum([int(s[0]) + int(s[1]) for s in score])

def pretty_score(score):
    return ','.join(['-'.join(s) for s in score])

def is_bagel(score):
    return all([not int(s[1]) for s in score])

score = ['76', '64']
print(get_total_games(score))
print(pretty_score(score))
print(is_bagel(score))
print(is_bagel(['60', '30']))

In [ ]:
def get_player_graph(player_id, prev_player_id=None):
    indent = '\t' if prev_player_id is not None else ''
    player = players.find_one(
        {'_id': str(player_id)},
        {'first_name': 1, 'last_name': 1, 'rating_level': 1}
    )
    print('{}{}: {} {} ({})'.format(
            indent, player['_id'], player['first_name'],
            player['last_name'], player['rating_level']
    ))
    counter = collections.Counter()
    query = {'$or': []}
    # sd_keys, wl_keys = ['singles', 'doubles'], ['winner', 'loser']
    sd_keys, wl_keys = ['singles'], ['winner', 'loser']
    for key in itertools.product(sd_keys, wl_keys):
        query['$or'].append({'.'.join(key): player_id})
    for match in matches.find(query):
        individual_match_found = False
        for sd in sd_keys:
            for individual_match in match[sd]:
                if individual_match_found:
                    break
                for iwl, wl in enumerate(wl_keys):
                    if individual_match[wl] == player_id or (
                        isinstance(individual_match[wl], list) and \
                        player_id in individual_match[wl]
                    ):
                        individual_match_found = True
                        score = individual_match['score']
                        opponent_id = individual_match[wl_keys[int(not(iwl))]]
                        if (
                            get_total_games(score) < 6 or # less than 1 set played
                            is_bagel(score) or # also skip bagels?
                            opponent_id is None or # default
                            opponent_id == prev_player_id # already added                         
                        ):
                            break
                        counter['{}.{}'.format(sd, wl)] += 1
                        print(
                            '{}{}:'.format(indent, match['_id']), wl[0].upper(),
                            pretty_score(score), 'vs', opponent_id
                        )
                        if prev_player_id is None:
                            # only go one level down
                            get_player_graph(opponent_id, prev_player_id=player_id)
                        break
    #print('{}{}'.format(indent, counter))


player_id = int(players.find_one({'last_name': 'Huck', 'first_name': 'Patrick'})['_id'])
get_player_graph(player_id)